In [1]:
!pip install brian2
!pip install -q condacolab
import condacolab
condacolab.install()
!conda create -y -n opensim_env python=3.10
!source /usr/local/etc/profile.d/conda.sh && conda activate opensim_env && conda install -y -c opensim-org opensim
!wget https://raw.githubusercontent.com/MathieuCharbonnier/Loop/refs/heads/main/activation.py
!wget https://raw.githubusercontent.com/MathieuCharbonnier/Loop/refs/heads/main/muscle_sim.py
!wget https://raw.githubusercontent.com/MathieuCharbonnier/Loop/refs/heads/main/neural_simulations.py
!wget https://raw.githubusercontent.com/MathieuCharbonnier/Loop/refs/heads/main/plot_time_series.py



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.3 MB/s eta 0:00:00
⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:13
🔁 Restarting kernel...
Channels:
 - conda-forge
Platform: linux-64
Solving environment: - \ done


==> WARNING: A newer version of conda exists. <==
    current version: 24.11.2
    latest version: 25.3.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local/envs/opensim_env

  added / updated specs:
    - python=3.10


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2025.1.31  |       hbcca054_0         154 KB  conda-forge
    ld_impl_linux-64-2.43      |       h712a8e2_4         656 K

In [5]:
!unzip Model.zip

Archive:  Model.zip
   creating: Model/
  inflating: Model/lumbar3.vtp       
  inflating: __MACOSX/Model/._lumbar3.vtp  
  inflating: Model/hat_jaw.vtp       
  inflating: __MACOSX/Model/._hat_jaw.vtp  
  inflating: Model/arm_r_4midph.vtp  
  inflating: __MACOSX/Model/._arm_r_4midph.vtp  
  inflating: Model/arm_r_5midph.vtp  
  inflating: __MACOSX/Model/._arm_r_5midph.vtp  
  inflating: Model/metacarpal4_lvs.vtp  
  inflating: __MACOSX/Model/._metacarpal4_lvs.vtp  
  inflating: Model/cerv4sm.vtp       
  inflating: __MACOSX/Model/._cerv4sm.vtp  
  inflating: Model/metatarsal1_rv.vtp  
  inflating: __MACOSX/Model/._metatarsal1_rv.vtp  
  inflating: Model/treadmill.vtp     
  inflating: __MACOSX/Model/._treadmill.vtp  
  inflating: Model/r_hand.vtp        
  inflating: __MACOSX/Model/._r_hand.vtp  
  inflating: Model/EES_opt_P1.ipynb  
  inflating: __MACOSX/Model/._EES_opt_P1.ipynb  
  inflating: Model/r_patella.vtp     
  inflating: __MACOSX/Model/._r_patella.vtp  
  inflating: Model/a

In [12]:
%load_ext autoreload
%autoreload 2

from brian2 import *
import numpy as np
import pandas as pd
import os
import subprocess
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d

from plot_time_series import plot_times_series
from neural_simulations import run_neural_simulations
from activation import decode_spikes_to_activation

In [27]:
t_stretching_afferent = 16*ms
t_motoneuron_muscle = 10*ms
t_reaction = t_stretching_afferent + t_motoneuron_muscle
n_loop = 10
dt = 0.1*ms
name_output = 'time_series.csv'
n = {"Ia": 10, "II": 10, "exc": 10, "motor": 10}

# Initialize stretch and velocity arrays
stretch_init = np.zeros(int(t_reaction/dt))
velocity_init = np.zeros(int(t_reaction/dt))
stretch = stretch_init
velocity = velocity_init

data = []
for j in range(1, n_loop):

    time = np.arange(j*t_reaction, (j+1)*t_reaction, dt)
    spikes_Ia, spikes_II,spikes_motoneuron= run_neural_simulations(stretch, velocity, neuron_population=n,dt=dt,T=t_reaction, w=500*uS, p=0.4 )
    activations=decode_spikes_to_activation(spikes_motoneuron, time)
    mean_activation = np.mean(activations, axis=0)

    # Run muscle simulation
    os.system(f"source /usr/local/etc/profile.d/conda.sh && conda activate opensim_env && python muscle_sim.py --time {time} --activation {mean_activation}")
    table = osim.TimeSeriesTable("muscle_state.sto")

    # Create batch data dictionary
    batch_data = {
        'time': time,
        **{f'spikes_times_II_{i}': time == spikes_II[i] for i in range(n['II'])},
        **{f'spikes_times_Ia_{i}': time == spikes_Ia[i] for i in range(n['Ia'])},
        **{f'spikes_times_motor_{i}': time == spikes_motoneuron[i] for i in range(n['motor'])},
        **{f'activation_motor_{i}': activations[i] for i in range(n['motor'])},
        'mean_activation': mean_activation,  # Fixed variable name
        'stretch': table.getDependentColumn('stretch').to_numpy(),  # Fixed table access
        'velocity': table.getDependentColumn('velocity').to_numpy(),
        'fiber_length': table.getDependentColumn('fiber_length').to_numpy(),
        'fiber_velocity': table.getDependentColumn('fiber_velocity').to_numpy()
    }

    data.append(batch_data)

# Create DataFrame and apply time shifts
df = pd.DataFrame(data)
time_shift = int(t_reaction/dt)
df['time'] = df['time'].shift(periods=time_shift).fillna(method='bfill')  # Fixed shift operation

# Select columns that contain spike times
spike_columns = [col for col in df.columns if 'spikes_times' in col]
time_shift_spikes = int(t_stretching_afferent/dt)

# Shift each of these columns by t_stretching_afferent
for col in spike_columns:
    df[col] = df[col].shift(periods=time_shift_spikes)

# Select the remaining columns
others_columns = [col for col in df.columns if col not in spike_columns]
for col in others_columns:
    df[col] = df[col].shift(periods=time_shift)  # Fixed parenthesis

# Set initial values
df.loc[:time_shift, ['stretch', 'velocity']] = [stretch_init, velocity_init]

# Save the DataFrame to CSV
df.to_csv(name_output, mode='a', header=False, index=False)

# Plot time series
plot_times_series(name_output)

WARNING    'dt' is an internal variable of group 'poissongroup_1', but also exists in the run namespace with the value 100. * usecond. The internal variable will be used. [brian2.groups.group.Group.resolve.resolution_conflict]
WARNING    'dt' is an internal variable of group 'poissongroup', but also exists in the run namespace with the value 100. * usecond. The internal variable will be used. [brian2.groups.group.Group.resolve.resolution_conflict]
WARNING    'w' is an internal variable of group 'synapses_1', but also exists in the run namespace with the value 0.5 * msiemens. The internal variable will be used. [brian2.groups.group.Group.resolve.resolution_conflict]
WARNING    'w' is an internal variable of group 'synapses_2', but also exists in the run namespace with the value 0.5 * msiemens. The internal variable will be used. [brian2.groups.group.Group.resolve.resolution_conflict]
WARNING    'w' is an internal variable of group 'synapses', but also exists in the run namespace with th

TypeError: Object of type <class 'dict'> does not have dimensions

In [28]:
    activations=decode_spikes_to_activation(spikes_motoneuron, time)

NameError: name 'second' is not defined